Ön hazırlık (kütüphane kurulumu)

In [ ]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.4 MB/s eta 0:00:00


Cirq ile Örnek

In [ ]:
import cirq
import numpy as np

# 2 qubit ve bir ancilla
qubits = cirq.LineQubit.range(2)

# Oracle (hedef: 11)
oracle = cirq.Circuit(
    cirq.Z(qubits[0]).controlled_by(qubits[1])
)

# Grover Diffusion Operator
def diffusion_operator(qubits):
    circuit = cirq.Circuit()
    circuit.append(cirq.H.on_each(*qubits))
    circuit.append(cirq.X.on_each(*qubits))
    circuit.append(cirq.H(qubits[-1]))
    circuit.append(cirq.Z(qubits[-1]).controlled_by(*qubits[:-1]))
    circuit.append(cirq.H(qubits[-1]))
    circuit.append(cirq.X.on_each(*qubits))
    circuit.append(cirq.H.on_each(*qubits))
    return circuit

# Başlangıç devresi (süperpozisyon)
circuit = cirq.Circuit()
circuit.append(cirq.H.on_each(*qubits))
circuit += oracle
circuit += diffusion_operator(qubits)

# Ölçüm
circuit.append(cirq.measure(*qubits, key='result'))

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)
print(result)


result=0101111000, 1110000111


Qiskit ile Örnek

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import GroverOperator

# 2 qubit
qc = QuantumCircuit(2)
qc.h([0,1])  # Süperpozisyon

# Oracle (hedef: |11>)
oracle = QuantumCircuit(2)
oracle.cz(0,1)

# Grover Diffusion
grover_op = GroverOperator(oracle)

# Uygula
qc.append(grover_op, [0,1])
qc.measure_all()

sim = Aer.get_backend('aer_simulator')
result = execute(qc, sim, shots=10).result()
print(result.get_counts())


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Örnek (Durum Vektörü Simülasyonu)

In [ ]:
import qutip as qt
import numpy as np

# Başlangıç: |00>
psi0 = qt.tensor(qt.basis(2,0), qt.basis(2,0))

# Hadamard uygulama
H = (1/np.sqrt(2)) * qt.Qobj([[1,1],[1,-1]])
H2 = qt.tensor(H,H)
psi = H2 * psi0

# Oracle (işaretleme |11>)
Z = qt.qeye(4)
Z[3,3] = -1
psi = Z * psi

# Diffusion operator
I = qt.qeye(4)
D = 2*psi*psi.dag() - I
psi = D * psi

# Son durum
print(psi)


TypeError: 'Qobj' object does not support item assignment